In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from joblib import Parallel, delayed
from keras.models import model_from_json
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

import constitutive
import displacement_solver
import gauss_pt_coord
import mesh_gen
import quadrature
import stress_gauss
import spr_stress
import patch_n_int_nodes
import stress_nodes_dc

2023-04-07 12:29:39.768748: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-07 12:29:40.384930: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-07 12:29:42.680151: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-07 12:29:42.714704: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-07 12:29:55.121290: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
# # 1) Inputs
# a) Problem parameters
# Domain geometry
domain_coord = np.array([[0, 0], [48, 44], [48, 60], [0, 44], [
                        24, 22], [48, 52], [24, 52], [0, 22]])
# Body force components
b = np.array([[0], [0]])
# Traction components
q = 1/16
T = np.array([[0, 0], [0, q], [0, 0]])
# b) Young's modulus
E = 1.0
# c) Poisson's ratio
nu = 1/3
# d) Problem type (0 ----> plane stress, 1 -----> plane strain)
problem_type = 0
# e) Element used for meshing (0 ---> 4 node quadrilateral, 1 -----> 8 node quadrilateral)
el_type_q4 = 0
el_type_q8 = 1
el_type_q9 = 2
# f) No. of Gauss points required for integration
ngp2d = 1
ngp1d = 2

# # g) Mesh sizes to be tested
N = [5]


In [3]:
# Q4 elements
u_list_q4 = Parallel(n_jobs=-1, verbose=100)(
    delayed(displacement_solver.solve_fem)(
        N[i], E, nu, ngp2d, ngp1d, el_type_q4, problem_type, domain_coord, b, T)
    for i in range(len(N)))

u_q4 = []
for i in range(len(N)):
    nx = N[i]
    ny = N[i]
    node_2d = np.arange((nx + 1) * (ny + 1)).reshape(nx + 1, ny + 1)
    node = node_2d[int(ny / 2), -1]
    dof = 2 * node + 1
    u_q4.append(u_list_q4[i][dof])
print(u_q4)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
(25, 8)
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    5.2s finished
[24.364007351422813]


In [4]:
ms=N[0]
u=u_list_q4[N.index(ms)]
nel=ms*ms  

In [5]:
C = constitutive.constitutive(E, nu, problem_type)
#nodal coordinates
nx = ms
ny = ms
el_type = 0
mesh_obj = mesh_gen.MeshGenerator(nx, ny, domain_coord.reshape(16, 1), el_type)
connect = mesh_obj.connectivity()
coord = mesh_obj.coord_array()


In [6]:
strs_ref=pd.read_csv('Data/mesh_18-143_model3_data.csv') 

In [7]:
strs_ref

,Unnamed: 0,x,y,sigma_x,sigma_y,sigma_xy
0,0,0.000000,0.000000,0.038969,0.012990,0.031987
1,1,2.666667,2.444444,0.042593,0.038722,0.036876
2,2,5.333333,4.888889,0.052135,0.033737,0.048589
3,3,8.000000,7.333333,0.059190,0.052859,0.055300
4,4,10.666667,9.777778,0.073768,0.045675,0.067387
...,...,...,...,...,...,...
65918,65918,46.657343,59.552448,-0.017590,0.112467,-0.093654
65919,65919,46.993007,59.664336,-0.203363,-0.154760,0.040485
65920,65920,47.328671,59.776224,-0.002240,0.142478,-0.103031
65921,65921,47.664336,59.888112,-0.266350,-0.214639,0.093489


In [8]:
var = 'x_y_xy'

In [9]:
X = strs_ref[['x', 'y']];
X.to_csv('Coord/mesh_18-143'+'.csv');
Y = strs_ref[['sigma_x', 'sigma_y', 'sigma_xy']];
Y

,sigma_x,sigma_y,sigma_xy
0,0.038969,0.012990,0.031987
1,0.042593,0.038722,0.036876
2,0.052135,0.033737,0.048589
3,0.059190,0.052859,0.055300
4,0.073768,0.045675,0.067387
...,...,...,...
65918,-0.017590,0.112467,-0.093654
65919,-0.203363,-0.154760,0.040485
65920,-0.002240,0.142478,-0.103031
65921,-0.266350,-0.214639,0.093489


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
scaled_x=scaler.transform(X)
scaled_test=scaler.transform(coord)

/home/krishna/Desktop/btp_499_code/btp_env/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [211]:
#ANN one model in whole domain
model1= Sequential()
model1.add(Dense(units=50, activation='relu'))
model1.add(Dense(units=44, activation='relu'))
model1.add(Dense(units=38, activation='relu'))
model1.add(Dense(units=31, activation='relu'))
model1.add(Dense(units=25, activation='relu'))
model1.add(Dense(units=18, activation='relu'))
model1.add(Dense(units=12, activation='relu'))
model1.add(Dense(units=5, activation='relu'))
model1.add(Dense(units=3,activation='linear'))
model1.compile(loss='mse', optimizer=tf.keras.optimizers.SGD(learning_rate=0.001))



In [212]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', verbose=1, patience=30)


In [213]:
np.random.seed(1)
history=model1.fit(x=scaled_x, y=Y, batch_size=256, epochs=300, verbose=1, validation_split=0.1,callbacks=[es])

Epoch 1/300
232/232 [==============================] - 9s 18ms/step - loss: 0.0043 - val_loss: 0.0087
Epoch 2/300
232/232 [==============================] - 3s 13ms/step - loss: 0.0040 - val_loss: 0.0082
Epoch 3/300
232/232 [==============================] - 3s 12ms/step - loss: 0.0039 - val_loss: 0.0079
Epoch 4/300
232/232 [==============================] - 3s 14ms/step - loss: 0.0038 - val_loss: 0.0077
Epoch 5/300
232/232 [==============================] - 3s 12ms/step - loss: 0.0038 - val_loss: 0.0077
Epoch 6/300
232/232 [==============================] - 3s 12ms/step - loss: 0.0037 - val_loss: 0.0076
Epoch 7/300
232/232 [==============================] - 3s 13ms/step - loss: 0.0037 - val_loss: 0.0076
Epoch 8/300
232/232 [==============================] - 3s 13ms/step - loss: 0.0036 - val_loss: 0.0075
Epoch 9/300
232/232 [==============================] - 3s 12ms/step - loss: 0.0035 - val_loss: 0.0074
Epoch 10/300
232/232 [==============================] - 3s 13ms/step - loss: 0.003

In [214]:
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('train_loss', fontsize=15)
# plt.ylabel('Loss', fontsize=15)
# plt.xlabel('Epoch', fontsize=15)
# plt.legend(['train', 'val'], loc='upper right')
# plt.xticks(fontsize=15)
# plt.yticks(fontsize=15)
# plt.show()
#plt.savefig("traintr.jpg")


In [215]:
# model_json = model1.to_json()
# with open('Saved_model/train_model3_ref_'+var+'.json', 'w') as json_file:
#     json_file.write(model_json)

# #serialize weights to HDF5 file
# model1.save_weights('Saved_model/train_model3_ref_'+var+'.h5')


In [216]:
 #Saving train model with different optimizer.
 model_json = model1.to_json()
 with open('Model/Model3/SGD_'+var+'.json', 'w') as json_file:
     json_file.write(model_json)

 #serialize weights to HDF5 file
 model1.save_weights('Model/Model3/SGD_'+var+'.h5')


In [217]:
# # load json and create model
# json_file = open('Saved_model/train_model3_ref_'+var+'.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)

# #load weights into new model
# loaded_model.load_weights('Saved_model/train_model3_ref_'+var+'.h5')


In [218]:
# Loading the Trained model of different Optimizer.

json_file = open('Model/Model3/SGD_'+var+'.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

#load weights into new model
loaded_model.load_weights('Model/Model3/SGD_'+var+'.h5')


In [219]:
#Stress calculation at Gauss points
stress = np.zeros((nel, ngp2d*ngp2d, 3))
strains = np.zeros((nel, ngp2d*ngp2d, 3))

for i in range(nel):
    stress_i_g = np.zeros((ngp2d*ngp2d, 3))
    strains_i_g = np.zeros((ngp2d*ngp2d, 3))

    stress_i_g, strains_i_g = stress_gauss.get_element_stress(
        i, ngp2d, el_type_q4, connect, coord, u, C)

    stress[i][:][:] = stress_i_g
    strains[i][:][:] = strains_i_g.reshape((1, 3))


In [220]:
# calcuation of gauss coordinates.
gauss_coords = np.zeros((nel, ngp2d*ngp2d, 2))
gp, weights = quadrature.quadrature(ngp2d)
for i in range(nel):
    node = connect[i, :]
    vertex_coord = coord[node, :].reshape(-1)
    gauss_coords[i][:][:] = gauss_pt_coord.gauss_pts(
        ngp2d, vertex_coord, gp, el_type)
gauss_coords = gauss_coords.reshape(gauss_coords.shape[0], -1)


In [221]:
#creation of patches for spr_stress;
patch, n_patches, int_nodes = patch_n_int_nodes.patch_n_int_nodes(ms)


In [222]:
#spr_stress STRESS Calculations
stress_spr = spr_stress.spr(gauss_coords, coord, connect, stress,
                            int_nodes, n_patches, patch, ms)
# stress_spr


In [223]:
#Directly calculated stress
stress_dc, strain_dc = stress_nodes_dc.stress_dc(
    connect, coord, u, nel, el_type, C)


In [224]:
# coord
# np.random.seed(1);
# pts = np.random.choice(np.arange(36),5);
# coord_test = coord[pts];
# coord_test
# stress_spr
total_nodes = (ms+1)**2


In [225]:
#outter points in
sp = []
# print(ms)
temp = ms+1
for i in range(ms-1):
    sp.append(temp)
    sp.append(temp+ms)
    temp = temp+ms+1
for i in range(ms+1):
    sp.append(temp)
    temp = temp+1

for i in range(ms+1):
    sp.append(i)

sp = sorted(sp)
# print(sp);


In [226]:
coord_corner = coord[sp]
# coord_corner


In [227]:
scaled_test_corner = scaler.transform(coord_corner)
# scaled_test_corner


/home/krishna/Desktop/btp_499_code/btp_env/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [228]:
ref_stress = pd.read_csv('Ref/ref_stress_for_ms_5_from_320.csv')
ref_stress = ref_stress.loc[:, ~ref_stress.columns.str.contains('^Unnamed')]
ref_stress = ref_stress.to_numpy()
# ref_stress


In [229]:
# a ---->reference stress
a = ref_stress.T
# print(len(a))


In [230]:
# a--->plane stress calculated from spr
b = stress_spr.T
print(len(b))


3


In [231]:
err_spr_abs = np.abs((a-b))
err_spr_perc = np.abs((a-b)/a)
err_dc_abs = np.abs((a-stress_dc.T))
err_dc_perc = np.abs((a-stress_dc.T)/a)
# err_spr_abs
# err_dc_abs


In [232]:
test = pd.DataFrame(scaled_test, columns=['a', 'b'])
# test


In [233]:
# c --->stress calculated using ANN trained model
c = loaded_model.predict(test)
len(c)


2/2 [==============================] - 1s 13ms/step


36

In [234]:
err_ann_abs = np.abs((a-c.T))
err_ann_perc = np.abs((a-c.T)/a)
# err_ann_abs


In [235]:

alll = np.concatenate([coord,
                       a.reshape(a.T.shape[0], 3),
                       b.reshape(b.T.shape[0], 3),
                       c.reshape(c.shape[0], 3),
                       err_spr_abs.reshape(err_spr_abs.T.shape[0], 3),
                       err_ann_abs.reshape(err_ann_abs.T.shape[0], 3),
                       err_dc_abs.reshape(err_dc_abs.T.shape[0], 3),
                       err_spr_perc.reshape(err_spr_perc.T.shape[0], 3),
                       err_ann_perc.reshape(err_ann_perc.T.shape[0], 3),
                       err_dc_perc.reshape(err_dc_perc.T.shape[0], 3)
                       ],
                      axis=1)
# alll


In [236]:
tdf = pd.DataFrame(alll, columns=['x',
                                  'y',
                                  'ref_stress_x',
                                  'ref_stress_y',
                                  'ref_stress_xy',
                                  'spr_stress_x',
                                  'spr_stress_y',
                                  'spr_stress_xy',
                                  'ann_stress_x',
                                  'ann_stress_y',
                                  'ann_stress_xy',
                                  'err_spr_abs_x',
                                  'err_spr_abs_y',
                                  'err_spr_abs_xy',
                                  'err_ann_abs_x',
                                  'err_ann_abs_y',
                                  'err_ann_abs_xy',
                                  'err_dc_abs_x',
                                  'err_dc_abs_y',
                                  'err_dc_abs_xy',
                                  'err_spr_perc_x',
                                  'err_spr_perc_y',
                                  'err_spr_perc_xy',
                                  'err_ann_perc_x',
                                  'err_ann_perc_y',
                                  'err_ann_perc_xy',
                                  'err_dc_perc_x',
                                  'err_dc_perc_y',
                                  'err_dc_perc_xy'
                                  ])
# print(tdf.head());


In [237]:
tdf.to_csv('mult_e_data/ms5/all_matlab_train_model3_ref_'+var+'.csv')
tdf.head()


,x,y,ref_stress_x,ref_stress_y,ref_stress_xy,spr_stress_x,spr_stress_y,spr_stress_xy,ann_stress_x,ann_stress_y,...,err_dc_abs_xy,err_spr_perc_x,err_spr_perc_y,err_spr_perc_xy,err_ann_perc_x,err_ann_perc_y,err_ann_perc_xy,err_dc_perc_x,err_dc_perc_y,err_dc_perc_xy
0,0.0,0.0,0.020590,0.066856,0.114837,0.063443,0.059761,0.057295,0.062336,0.021496,...,0.003450,2.081206,0.106128,0.501073,2.027480,0.304035,0.056023,2.244944,0.033750,0.030039
1,9.6,8.8,0.142259,0.142736,0.116694,0.060733,0.062001,0.065805,0.087182,0.043291,...,0.285664,0.573084,0.565624,0.436090,0.168746,0.333842,0.545972,0.489825,0.133649,2.447965
2,19.2,17.6,0.073042,0.093437,0.099048,0.063212,0.086114,0.096941,0.108403,0.072166,...,0.017992,0.134571,0.078366,0.021269,0.017998,0.014087,0.053997,0.192864,0.120837,0.181649
3,28.8,26.4,0.094280,0.076036,0.014692,0.093381,0.065664,0.014115,0.118254,0.102980,...,0.006585,0.009532,0.136410,0.039297,0.005486,0.170489,1.098081,0.152581,0.531915,0.448163
4,38.4,35.2,0.080207,0.072637,0.051343,0.027299,0.068231,0.050842,0.095084,0.114050,...,0.007476,0.659641,0.060657,0.009764,0.215882,0.098260,0.030337,0.028527,0.229736,0.145609


In [238]:
tdf['err_spr_perc_x'].mean()


0.6860461513578601

In [239]:
tdf['err_spr_perc_y'].mean()


0.32947246179473

In [240]:
tdf['err_spr_perc_xy'].mean()


0.5658925933085241

In [241]:
tdf['err_ann_perc_x'].mean()


0.871329850035049

In [242]:
tdf['err_ann_perc_y'].mean()


0.9084175157705499

In [243]:
tdf['err_ann_perc_xy'].mean()

0.834772850789505